In [2]:
import time
import math
import sympy as sp
import numpy as np
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler 

In [3]:
def balance(x, q):
    return np.array([n if n <= (q-1)/2 else n-q for n in x.flatten()]).reshape(np.shape(x))

In [4]:
class commitmentScheme(object):
    lamb = None
    p = None
    n = None
    k = None 
    m = None
    l = None
    v = None
    N = None
    B = None
    
    def __init__(self, lamb, p, n, k, m):
        self.lamb = lamb
        self.p = p
        self.n = n 
        self.k = k 
        self.m = m
        
    def gen(self):
        #Set parameters as in BBC+18
        P = self.n*self.k**2*self.m**2*self.p**2
        B = P*self.k*self.m*self.n #B = O(PN)
        Pp = B*P
        r = int(np.ceil(np.log(self.n*self.k*self.m))) #r = O(log n) or O(log N)? 
        q = sp.nextprime(Pp*np.sqrt(r))
        print("P = " + str(P))
        print("B = " + str(B))
        print("Pp = " + str(Pp))
        print("r = " + str(r))
        print("log2 q = " + str(math.log2(q)))
        
        #n = int(np.ceil(np.sqrt(self.N*r*math.log(q, self.p))))
        #k = int(np.ceil(self.lamb/np.log2(self.p)))
        #m = int(np.ceil(self.N/(n*k)))
        
        A1 = np.array([randint((-q+1)//2, (q-1)//2) for _ in range(r*int(2*r*math.log(q, self.p)))]).reshape(r, int(2*r*math.log(q, self.p))) 
        A2 = np.array([randint((-q+1)//2, (q-1)//2) for _ in range(r*self.n)]).reshape(r, self.n)
        print("Shapes of matrices A1, A2: " + str(A1.shape) + " " + str(A2.shape))
        return (self.p, q, r, A1, A2)
    
def commit(msg, r, ck): #r = randomness
    q = ck[1]
    A1 = ck[3]
    A2 = ck[4]
    C = np.matmul(A1, r[0])%q + np.matmul(A2, msg[0])%q
    for i in range(msg.shape[0]-1):
        C = np.vstack((C, np.matmul(A1, r[i+1])%q + np.matmul(A2, msg[i+1])%q))
    return balance(C, q)
        #return C

In [5]:
def commit_test():
    
    #Commitment for k messages msg in (Z_p)^n
    #Can be seen as a commitment for a (nxk)-matrix
    #In the arithmetic circuit protocol we will commit to m (nxk)-matrices 
    
    lamb = 256
    p = 4093
    #N = 10000
    n = 10000
    k = int(np.ceil(lamb/np.log2(p)))
    m = 100 #int(N/(k*n))
    
    N = m*k*n
    print(N)
    
    sigma1 = 48*np.sqrt(k*n)*k*m*p**2
    
    D = DiscreteGaussianDistributionIntegerSampler(sigma1)
    
    msg = np.random.randint((-p+1)/2, (p-1)/2+1, size = [k, n])
    
    print("Messages: " + str(msg) + " (modulo " + str(p) + ")")
    
    start_time = time.time()

    cs = commitmentScheme(lamb, p, n, k, m)

    ck = cs.gen()
    
    r = np.array([D() for _ in range(k*int(2*ck[2]*math.log(ck[1], ck[0])))]).reshape(k, int(2*ck[2]*math.log(ck[1], ck[0])))

    com_msg = commit(msg, r, ck)

    end_time = time.time()

    print("Committed messages: " + str(com_msg) + " (modulo " + str(ck[1]) + ")")
    
    print("Total execution time: " + str(end_time - start_time) + " seconds")

commit_test()

2200000
Messages: [[  -20   239 -1629 ...  1852   352   782]
 [  186  1831  -301 ...  1947   313 -1193]
 [  386  -400  1747 ... -1098 -1597 -1478]
 ...
 [ 1335 -1496  1493 ...  1550  -502   876]
 [-1714   599  1975 ...  1448 -1625  -975]
 [-1898   529 -1319 ...   509   318  1317]] (modulo 4093)
P = 81082821160000000
B = 178382206552000000000000
Pp = 14463732551981996240320000000000000000000
r = 15
log2 q = 135.36300899962296
Shapes of matrices A1, A2: (15, 338) (15, 1000)
Committed messages: [[43942240552108980986839528272558465999014
  39857391881716726472198733334277786883990
  -13229959856418066981152910542474257410321
  13982089063357815007292927543429233817712
  7754775984155981600606797076062795168085
  9147000900089259571505434626428833544906
  25324009011263953109790451557184723524835
  26857539590674168653524963170357969010123
  -25432927677873293637242291767242697953807
  3392434102167799603853536654308654959371
  -6703864732416371355745992270697884798571
  170359241885995339